- read datasets, potentially using sql query if it doesn't take long
- do my own basic explorations, compare to Hauke's
- Visualize labels
- IoU

In [ ]:
# Imports
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Read dataset

In [ ]:
### Functions from https://github.com/zooniverse/Data-digging/blob/master/example_scripts/astronomy_rewind/workflow1to2.py

def JSONParser(data):
    """call json.loads"""
    return json.loads(data)


def load_classifications(filename, json_columns=None):
    """
    Load classifications into pandas dataframe.
    Some columns of the csv are embedded json and need special parsing.
    """
    json_columns = json_columns or ['metadata', 'annotations', 'subject_data']
    converters = {i: JSONParser for i in json_columns}

    return pd.read_csv(filename, converters=converters)

def unpack(series):
    """
    Return the first value in a series.
    All annotations values are lists because of a few multiple tasks.
    The second multiple task always has the value of 'None of the above'
    (For this dataset!)
    """
    return [a[0] for a in series]


def parse_classifications(filename,**kwarg):
    """
    Load classifications and datamunge annotations column.
    """
    data = load_classifications(filename,**kwarg)

    # Only need the first item in the annotations list of json objects
    data['annotations'] = unpack(data['annotations'])
    return data

In [ ]:
clas_fn = '../zooniverse_raw/sugar-flower-fish-or-gravel-classifications_18_11_02.csv'
subj_fn = '../zooniverse_raw/sugar-flower-fish-or-gravel-subjects_18_11_05.csv'

In [ ]:
clas = parse_classifications(clas_fn,json_columns=['metadata', 'annotations', 'subject_data'])
subj = load_classifications(subj_fn); subj.head()

In [ ]:
subj

In [ ]:
subj.tail()

In [ ]:
subj['metadata'][0]['fn']

In [ ]:
subjects_name = {60811:'BCO_DJF_Aqua',60812:'BCO_DJF_Terra',60813: 'BCO_MAM_Aqua',\
                 60814: 'BCO_MAM_Terra',60815:'R2_DJF_Aqua',60816:'R2_DJF_Terra',\
                 60817:'R3_DJF_Aqua', 60818:'R3_DJF_Terra',60819:'R3_SON_Aqua',\
                 60835: 'R3_SON_Terra'}

In [ ]:
subj2fn = {}
for i, r in subj.iterrows():
    if r['subject_set_id'] in list(subjects_name.keys()):
        subj2fn[r['subject_id']] =  r['metadata']['fn']

In [ ]:
len(subj2fn)

In [ ]:
len(clas)

In [ ]:
clas.head()

In [ ]:
clas.tail()

In [ ]:
prac = clas[clas['workflow_name'] == 'Practice']

In [ ]:
len(prac)

In [ ]:
clas['num_boxes'] = clas['annotations'].apply(lambda x: len(x['value']))

In [ ]:
clas['num_boxes'].describe()

In [ ]:
clas['num_boxes'].sum()

In [ ]:
clas['num_boxes'].hist(bins=np.arange(13))

In [ ]:
clas['user_name'].unique(), clas['user_name'].unique().shape

In [ ]:
clas['user_name'].value_counts()

In [ ]:
prac['user_name'].value_counts()

## Basic exploration

### Distribution of labels

In [ ]:
labels = pd.Series([l['tool_label'] for a in clas['annotations'] for l in a['value'] ])

In [ ]:
len(labels)

In [ ]:
labels.value_counts()

## Visualize labels

In [ ]:
clas.head()

In [ ]:
full = clas[clas['workflow_name'] == 'Full dataset']

In [ ]:
full.drop(full.index[0], inplace=True)

In [ ]:
full['datetime'] = pd.to_datetime(full['created_at'])

In [ ]:
full = full[full['datetime'].dt.date == np.datetime64('2018-11-02')]

In [ ]:
full

In [ ]:
full['fn'] = full.apply(lambda r: subj2fn[r['subject_ids']], axis=1)

In [ ]:
full.head()

In [ ]:
from PIL import Image

In [ ]:
full['fn'].iloc[0]

In [ ]:
from collections import defaultdict

In [ ]:
subjects = list(subj2fn.keys())

In [ ]:
full

In [ ]:
annos = defaultdict(list)
for i, r in full.iterrows():
    for a in r['annotations']['value']:
        annos[r['subject_ids']].append([
            r['user_name'],
            [a['x'], a['y'], a['width'], a['height']],
            a['tool_label']
        ])

In [ ]:
annos

In [ ]:
s = 27146483

In [ ]:
img = Image.open(subj2fn[s])

In [ ]:
annos[s]

In [ ]:
annos[s][0][1]

In [ ]:
import matplotlib.patches as patches

In [ ]:
s = subjects[5009]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
img = Image.open(subj2fn[s])
ax.imshow(img)
for a in annos[s]:
    coords = a[1]
    rect = patches.Rectangle((coords[0],coords[1]), coords[2], coords[2], facecolor='none', edgecolor='r')
    ax.add_patch(rect)
    ax.text(coords[0],coords[1], a[2] + ' - ' + a[0], color='red', fontsize=15, va='top')

In [ ]:
labels = pd.Series([l['tool_label'] for a in clas['annotations'] for l in a['value'] ])

In [ ]:
full['annotations'].iloc[0]